In [1]:
import numpy as np
import pandas as pd

np.random.seed(42)
filtered_df = pd.read_csv("../processed_data/bond_data_normalized_w_senti.csv")

# 确保 '日期' 列是 datetime 格式
filtered_df["日期"] = pd.to_datetime(filtered_df["日期"], format="%Y-%m-%d")

# 按 '日期' 和 '发行人中文简称' 排序，确保数据有序
filtered_df = filtered_df.sort_values(by=["发行人中文简称", "日期"])

# 设定多级索引，以 '日期' 和 '发行人中文简称' 共同唯一标识每一行
filtered_df = filtered_df.set_index(["日期", "发行人中文简称"])

# 获取 t+1、t+2、t+3 的因变量（后一天、后两天、后三天的 '风险价差'）
filtered_df["y_t1"] = filtered_df.groupby("发行人中文简称")["到期收益率"].shift(-1)
filtered_df["y_t2"] = filtered_df.groupby("发行人中文简称")["到期收益率"].shift(-2)
filtered_df["y_t3"] = filtered_df.groupby("发行人中文简称")["到期收益率"].shift(-3)

# 先确定完整的索引
valid_indices = filtered_df[["y_t1", "y_t2", "y_t3"]].notna().all(axis=1)  # 确保所有因变量均非空

# 过滤出完整数据
filtered_df = filtered_df.loc[valid_indices]

# # 提取自变量
# X_A = filtered_df.iloc[:, 1:49]  # 第1到第48列
# X_B = filtered_df.iloc[:, list(range(1, 49)) + [56]]  # 第1到第48列加上 sentiment（第56列）
# X_A = X_A.reset_index()
# X_B = X_B.reset_index()

# 提取因变量
y_t1 = filtered_df["y_t1"]
y_t2 = filtered_df["y_t2"]
y_t3 = filtered_df["y_t3"]

# 重置索引，使 '日期' 和 '发行人中文简称' 成为普通列
X_A = filtered_df.reset_index().iloc[:, 2:50]
X_B = filtered_df.reset_index().iloc[:, list(range(2, 50)) + [56]]
y_t1 = y_t1.reset_index(drop=True)
y_t2 = y_t2.reset_index(drop=True)
y_t3 = y_t3.reset_index(drop=True)

# 检查数据格式
print(X_A.head())
print(X_B.head())
print(y_t1.head(), y_t2.head(), y_t3.head())



   中间价:美元兑人民币  Shibor:3月    制造业PMI  宏观经济景气指数:先行指数  PPI:当月同比  CPI:当月同比  \
0   -1.225054   0.304557 -0.130325      -0.545212 -1.781044 -0.290924   
1   -1.238866   0.304557 -0.130325      -0.545212 -1.781044 -0.290924   
2   -1.241233   0.292076 -0.130325      -0.545212 -1.781044 -0.290924   
3   -1.242417   0.292076 -0.130325      -0.545212 -1.781044 -0.290924   
4   -1.243601   0.292076 -0.130325      -0.545212 -1.781044 -0.290924   

   GDP:不变价:当季同比  社会融资规模存量:期末同比  所属申万一级行业指数  债券分类违约概率  ...   应收账款周转率   流动资产周转率  \
0      0.311153       0.059156    1.892971 -0.912128  ... -0.024039 -0.172297   
1      0.311153       0.059156    1.912229 -0.912128  ... -0.024039 -0.172297   
2      0.311153       0.059156    1.958379 -0.912128  ... -0.024039 -0.172297   
3      0.311153       0.059156    1.711148 -0.912128  ... -0.024039 -0.172297   
4      0.311153       0.059156    1.772551 -0.912128  ... -0.024039 -0.172297   

   股东权益周转率    总资产周转率     授信剩余率    授信环比变动     担保授信比    同期国债利率       成交量  \


In [2]:
print(len(X_A), len(X_B), len(y_t1), len(y_t2), len(y_t3))
print(type(X_A), type(X_B), type(y_t1), type(y_t2), type(y_t3))

4760928 4760928 4760928 4760928 4760928
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'> <class 'pandas.core.series.Series'>


In [3]:
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
# from scipy.stats import ttest_ind
# from tqdm import tqdm

# targets = [y_t1, y_t2, y_t3]
# # 存储结果的 DataFrame
# performance_results = []
# t_test_pred_results = []
# t_test_R2_results = []
# t_test_MSE_results = []
# t_test_MAE_results = []
# t_test_RMSE_results = []
# feature_importance_A = []
# feature_importance_B = []

# # 遍历 t+1, t+2, t+3
# for target in tqdm(targets):
#     y = target  # 获取当前的因变量

#     # 拆分数据集
#     X_A_train, X_A_test, y_train, y_test = train_test_split(X_A, y, test_size=0.2, random_state=42)
#     X_B_train, X_B_test = train_test_split(X_B, test_size=0.2, random_state=42)[0:2]  # 仅拆分 X_B

#     # 训练随机森林模型（实验 A）
#     rf_A = RandomForestRegressor(random_state=42)
#     rf_A.fit(X_A_train, y_train)
#     y_pred_A = rf_A.predict(X_A_test)

#     # 训练随机森林模型（实验 B）
#     rf_B = RandomForestRegressor(random_state=42)
#     rf_B.fit(X_B_train, y_train)
#     y_pred_B = rf_B.predict(X_B_test)

#     # 计算性能指标
#     r2_A = r2_score(y_test, y_pred_A)
#     mse_A = mean_squared_error(y_test, y_pred_A)
#     mae_A = mean_absolute_error(y_test, y_pred_A)
#     rmse_A = mean_squared_error(y_test, y_pred_A, squared=False)

#     r2_B = r2_score(y_test, y_pred_B)
#     mse_B = mean_squared_error(y_test, y_pred_B)
#     mae_B = mean_absolute_error(y_test, y_pred_B)
#     rmse_B = mean_squared_error(y_test, y_pred_B, squared=False)

#     # 存储性能差异
#     performance_results.append({
#         "Time Horizon": target,
#         "Metric": "R²",
#         "Experiment A": r2_A,
#         "Experiment B": r2_B,
#         "Improvement": r2_B - r2_A
#     })
#     performance_results.append({
#         "Time Horizon": target,
#         "Metric": "MSE",
#         "Experiment A": mse_A,
#         "Experiment B": mse_B,
#         "Improvement": mse_A - mse_B
#     })
#     performance_results.append({
#         "Time Horizon": target,
#         "Metric": "MAE",
#         "Experiment A": mae_A,
#         "Experiment B": mae_B,
#         "Improvement": mae_A - mae_B
#     })
#     performance_results.append({
#         "Time Horizon": target,
#         "Metric": "RMSE",
#         "Experiment A": rmse_A,
#         "Experiment B": rmse_B,
#         "Improvement": rmse_A - rmse_B
#     })

#     # t 检验
#     t_stat_pred, p_value_pred = ttest_ind(y_pred_A, y_pred_B)
#     t_test_pred_results.append({
#         "Time Horizon": target,
#         "T-statistic": t_stat_pred,
#         "P-value": p_value_pred,
#         "Significant": "Yes" if p_value_pred < 0.05 else "No"
#     })

#     t_stat_R2, p_value_R2 = ttest_ind(r2_A, r2_B)
#     t_test_R2_results.append({
#         "Time Horizon": target,
#         "T-statistic": t_stat_R2,
#         "P-value": p_value_R2,
#         "Significant": "Yes" if p_value_pred < 0.05 else "No"
#     })

#     t_stat_MSE, p_value_MSE = ttest_ind(mse_A, mse_B)
#     t_test_MSE_results.append({
#         "Time Horizon": target,
#         "T-statistic": t_stat_MSE,
#         "P-value": p_value_MSE,
#         "Significant": "Yes" if p_value_pred < 0.05 else "No"
#     })

#     t_stat_MAE, p_value_MAE = ttest_ind(mae_A, mae_B)
#     t_test_MAE_results.append({
#         "Time Horizon": target,
#         "T-statistic": t_stat_MAE,
#         "P-value": p_value_MAE,
#         "Significant": "Yes" if p_value_pred < 0.05 else "No"
#     })

#     t_stat_RMSE, p_value_RMSE = ttest_ind(rmse_A, rmse_B)
#     t_test_RMSE_results.append({
#         "Time Horizon": target,
#         "T-statistic": t_stat_RMSE,
#         "P-value": p_value_RMSE,
#         "Significant": "Yes" if p_value_pred < 0.05 else "No"
#     })

#     # 特征重要性
#     importance_A = rf_A.feature_importances_
#     importance_B = rf_B.feature_importances_

#     feature_importance_A.append(pd.DataFrame({"Feature": X_A.columns, "Importance": importance_A}).sort_values(by="Importance", ascending=False).assign(Time_Horizon=target))
#     feature_importance_B.append(pd.DataFrame({"Feature": X_B.columns, "Importance": importance_B}).sort_values(by="Importance", ascending=False).assign(Time_Horizon=target))

# # 转换为 DataFrame
# performance_df = pd.DataFrame(performance_results)
# t_test_pred_results_df = pd.DataFrame(t_test_pred_results)
# t_test_R2_results_df = pd.DataFrame(t_test_R2_results)
# t_test_MSE_results_df = pd.DataFrame(t_test_MSE_results)
# t_test_MAE_results_df = pd.DataFrame(t_test_MAE_results)
# importance_df_A = pd.concat(feature_importance_A)
# importance_df_B = pd.concat(feature_importance_B)

# print(performance_df)
# print(t_test_pred_results_df)
# print(t_test_R2_results_df)
# print(t_test_MSE_results_df)
# print(t_test_MAE_results_df)
# print(importance_df_A)
# print(importance_df_B)

In [4]:
from tqdm import tqdm
import cudf
import cupy as cp
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestRegressor
from cuml.metrics import r2_score, mean_squared_error, mean_absolute_error
from cupyx.scipy import special

def cupy_ttest_ind(a, b):
    # 确保输入是 cupy 数组
    if isinstance(a, float):
        a = cp.array([a])  # 转换为 cupy 数组
    if isinstance(b, float):
        b = cp.array([b])

    mean_a, mean_b = cp.mean(a), cp.mean(b)
    var_a, var_b = cp.var(a, ddof=1), cp.var(b, ddof=1)
    n_a, n_b = a.size, b.size

    pooled_se = cp.sqrt(var_a / n_a + var_b / n_b)
    t_stat = (mean_a - mean_b) / pooled_se
    df = (var_a / n_a + var_b / n_b) ** 2 / ((var_a / n_a) ** 2 / (n_a - 1) + (var_b / n_b) ** 2 / (n_b - 1))

    # 计算 p 值
    p_value = 2 * special.betainc(df / 2, 0.5, df / (df + t_stat**2))

    return t_stat, p_value

def permutation_importance(model, X, y, feature_names=None, metric=mean_squared_error, n_repeats=5):
    """
    计算基于置换的重要性
    参数:
    - model: 训练好的 cuML RandomForestRegressor
    - X: 特征矩阵 (CuPy array)
    - y: 目标值 (CuPy array)
    - feature_names: 特征名称 (列表或 cudf.Index)
    - metric: 评估指标 (默认 MSE)
    - n_repeats: 每个特征置换的重复次数
    
    返回:
    - feature_importances: 包含特征名和重要性的 cudf.DataFrame
    """
    # 确保 X 是 cupy.ndarray
    if isinstance(X, cudf.DataFrame):
        feature_names = X.columns  # 获取特征名称
        X = X.to_cupy()
    
    baseline_error = metric(y, model.predict(X))  # 计算基准 MSE
    importances = cp.zeros(X.shape[1])

    for col in range(X.shape[1]):
        scores = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[:, col] = X[:, col][cp.random.permutation(X.shape[0])]  # 置换该列
            permuted_error = metric(y, model.predict(X_permuted))
            scores.append(permuted_error - baseline_error)  # 误差增加量
        
        importances[col] = cp.mean(cp.array(scores))  # 计算均值
    
    # 确保特征名存在
    if feature_names is None:
        feature_names = [f"Feature {i}" for i in range(X.shape[1])]

    # 创建 cudf.DataFrame 存储特征重要性
    feature_importances = cudf.DataFrame({
        "Feature": feature_names,
        "Importance": importances.get()
    }).sort_values(by="Importance", ascending=False)  # 按重要性降序排列

    return feature_importances

cp.cuda.Device(0).use()
X_A = cudf.DataFrame.from_pandas(X_A)
X_B = cudf.DataFrame.from_pandas(X_B)
y = y_t1

performance_results = []
t_test_results = []

y = cp.array(y)
# 拆分数据集
X_A_train, X_A_test, y_train, y_test = train_test_split(X_A, y, test_size=0.2, random_state=42)
X_B_train, X_B_test = train_test_split(X_B, test_size=0.2, random_state=42)[0:2]  # 仅拆分 X_B

# 训练随机森林模型（实验 A）
rf_A = RandomForestRegressor(random_state=42)
rf_A.fit(X_A_train, y_train)
y_pred_A = rf_A.predict(X_A_test)

# 训练随机森林模型（实验 B）
rf_B = RandomForestRegressor(random_state=42)
rf_B.fit(X_B_train, y_train)
y_pred_B = rf_B.predict(X_B_test)

y_pred_A = cp.array(y_pred_A)
y_pred_B = cp.array(y_pred_B)

# 计算性能指标
r2_A = r2_score(y_test, y_pred_A)
mse_A = mean_squared_error(y_test, y_pred_A)
mae_A = mean_absolute_error(y_test, y_pred_A)
rmse_A = cp.sqrt(mse_A)

r2_B = r2_score(y_test, y_pred_B)
mse_B = mean_squared_error(y_test, y_pred_B)
mae_B = mean_absolute_error(y_test, y_pred_B)
rmse_B = cp.sqrt(mse_B)

# 存储性能差异
performance_results.append({
    "Time Horizon": "y_t1",
    "Metric": "R²",
    "Experiment A": r2_A,
    "Experiment B": r2_B,
    "Improvement": r2_B - r2_A
})
performance_results.append({
    "Time Horizon": "y_t1",
    "Metric": "MSE",
    "Experiment A": mse_A,
    "Experiment B": mse_B,
    "Improvement": mse_B - mse_A
})
performance_results.append({
    "Time Horizon": "y_t1",
    "Metric": "MAE",
    "Experiment A": mae_A,
    "Experiment B": mae_B,
    "Improvement": mae_B - mae_A
})
performance_results.append({
    "Time Horizon": "y_t1",
    "Metric": "RMSE",
    "Experiment A": rmse_A,
    "Experiment B": rmse_B,
    "Improvement": rmse_B - rmse_A
})

# t 检验
t_stat_pred, p_value_pred = cupy_ttest_ind(y_pred_A, y_pred_B)
t_test_results.append({
    "Time Horizon": "y_t1",
    "t-targe": "Prediction",
    "T-statistic": t_stat_pred,
    "P-value": p_value_pred,
    "Significant": "Yes" if p_value_pred < 0.05 else "No"
})


# 特征重要性
feature_importance_A = permutation_importance(rf_A, X_A_test, y_test)
feature_importance_B = permutation_importance(rf_B, X_B_test, y_test)

for entry in performance_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float

for entry in t_test_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float


# 转换为 DataFrame
performance_df = cudf.DataFrame(performance_results)
t_test_results_df = cudf.DataFrame(t_test_results)

print(performance_df)
print(t_test_results_df)
print(feature_importance_A)
print(feature_importance_B)

/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


  Time Horizon Metric  Experiment A  Experiment B  Improvement
0         y_t1     R²      0.197879      0.186903    -0.010975
1         y_t1    MSE      1.306911      1.324794     0.017882
2         y_t1    MAE      0.033187      0.033535     0.000349
3         y_t1   RMSE      1.143202      1.150997     0.007795
  Time Horizon     t-targe  T-statistic   P-value Significant
0         y_t1  Prediction    -0.172738  1.725714          No
          Feature  Importance
47           剩余期限    0.147290
19           负债合计    0.088375
30     有形净值债务率(%)    0.079406
13           利润总额    0.078049
23        筹资活动现金流    0.070877
20         股东权益合计    0.064742
33       资产净利率(%)    0.058499
35    平均净资产收益率(%)    0.054455
22        投资活动现金流    0.050891
5        CPI:当月同比    0.044609
9        债券分类违约概率    0.043814
32       销售净利率(%)    0.041444
24           总现金流    0.039484
3   宏观经济景气指数:先行指数    0.029133
40        股东权益周转率    0.028696
2          制造业PMI    0.028387
28       资产负债率(%)    0.028297
1       Shibor:3月    

In [5]:
from tqdm import tqdm
import cudf
import cupy as cp
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestRegressor
from cuml.metrics import r2_score, mean_squared_error, mean_absolute_error
from cupyx.scipy import special

def cupy_ttest_ind(a, b):
    # 确保输入是 cupy 数组
    if isinstance(a, float):
        a = cp.array([a])  # 转换为 cupy 数组
    if isinstance(b, float):
        b = cp.array([b])

    mean_a, mean_b = cp.mean(a), cp.mean(b)
    var_a, var_b = cp.var(a, ddof=1), cp.var(b, ddof=1)
    n_a, n_b = a.size, b.size

    pooled_se = cp.sqrt(var_a / n_a + var_b / n_b)
    t_stat = (mean_a - mean_b) / pooled_se
    df = (var_a / n_a + var_b / n_b) ** 2 / ((var_a / n_a) ** 2 / (n_a - 1) + (var_b / n_b) ** 2 / (n_b - 1))

    # 计算 p 值
    p_value = 2 * special.betainc(df / 2, 0.5, df / (df + t_stat**2))

    return t_stat, p_value

def permutation_importance(model, X, y, feature_names=None, metric=mean_squared_error, n_repeats=5):
    """
    计算基于置换的重要性
    参数:
    - model: 训练好的 cuML RandomForestRegressor
    - X: 特征矩阵 (CuPy array)
    - y: 目标值 (CuPy array)
    - feature_names: 特征名称 (列表或 cudf.Index)
    - metric: 评估指标 (默认 MSE)
    - n_repeats: 每个特征置换的重复次数
    
    返回:
    - feature_importances: 包含特征名和重要性的 cudf.DataFrame
    """
    # 确保 X 是 cupy.ndarray
    if isinstance(X, cudf.DataFrame):
        feature_names = X.columns  # 获取特征名称
        X = X.to_cupy()
    
    baseline_error = metric(y, model.predict(X))  # 计算基准 MSE
    importances = cp.zeros(X.shape[1])

    for col in range(X.shape[1]):
        scores = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[:, col] = X[:, col][cp.random.permutation(X.shape[0])]  # 置换该列
            permuted_error = metric(y, model.predict(X_permuted))
            scores.append(permuted_error - baseline_error)  # 误差增加量
        
        importances[col] = cp.mean(cp.array(scores))  # 计算均值
    
    # 确保特征名存在
    if feature_names is None:
        feature_names = [f"Feature {i}" for i in range(X.shape[1])]

    # 创建 cudf.DataFrame 存储特征重要性
    feature_importances = cudf.DataFrame({
        "Feature": feature_names,
        "Importance": importances.get()
    }).sort_values(by="Importance", ascending=False)  # 按重要性降序排列

    return feature_importances

cp.cuda.Device(0).use()
X_A = cudf.DataFrame.from_pandas(X_A)
X_B = cudf.DataFrame.from_pandas(X_B)
y = y_t2

performance_results = []
t_test_results = []

y = cp.array(y)
# 拆分数据集
X_A_train, X_A_test, y_train, y_test = train_test_split(X_A, y, test_size=0.2, random_state=42)
X_B_train, X_B_test = train_test_split(X_B, test_size=0.2, random_state=42)[0:2]  # 仅拆分 X_B

# 训练随机森林模型（实验 A）
rf_A = RandomForestRegressor(random_state=42)
rf_A.fit(X_A_train, y_train)
y_pred_A = rf_A.predict(X_A_test)

# 训练随机森林模型（实验 B）
rf_B = RandomForestRegressor(random_state=42)
rf_B.fit(X_B_train, y_train)
y_pred_B = rf_B.predict(X_B_test)

y_pred_A = cp.array(y_pred_A)
y_pred_B = cp.array(y_pred_B)

# 计算性能指标
r2_A = r2_score(y_test, y_pred_A)
mse_A = mean_squared_error(y_test, y_pred_A)
mae_A = mean_absolute_error(y_test, y_pred_A)
rmse_A = cp.sqrt(mse_A)

r2_B = r2_score(y_test, y_pred_B)
mse_B = mean_squared_error(y_test, y_pred_B)
mae_B = mean_absolute_error(y_test, y_pred_B)
rmse_B = cp.sqrt(mse_B)

# 存储性能差异
performance_results.append({
    "Time Horizon": "y_t2",
    "Metric": "R²",
    "Experiment A": r2_A,
    "Experiment B": r2_B,
    "Improvement": r2_B - r2_A
})
performance_results.append({
    "Time Horizon": "y_t2",
    "Metric": "MSE",
    "Experiment A": mse_A,
    "Experiment B": mse_B,
    "Improvement": mse_B - mse_A
})
performance_results.append({
    "Time Horizon": "y_t2",
    "Metric": "MAE",
    "Experiment A": mae_A,
    "Experiment B": mae_B,
    "Improvement": mae_B - mae_A
})
performance_results.append({
    "Time Horizon": "y_t2",
    "Metric": "RMSE",
    "Experiment A": rmse_A,
    "Experiment B": rmse_B,
    "Improvement": rmse_B - rmse_A
})

# t 检验
t_stat_pred, p_value_pred = cupy_ttest_ind(y_pred_A, y_pred_B)
t_test_results.append({
    "Time Horizon": "y_t2",
    "t-targe": "Prediction",
    "T-statistic": t_stat_pred,
    "P-value": p_value_pred,
    "Significant": "Yes" if p_value_pred < 0.05 else "No"
})


# 特征重要性
feature_importance_A = permutation_importance(rf_A, X_A_test, y_test)
feature_importance_B = permutation_importance(rf_B, X_B_test, y_test)

for entry in performance_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float

for entry in t_test_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float


# 转换为 DataFrame
performance_df = cudf.DataFrame(performance_results)
t_test_results_df = cudf.DataFrame(t_test_results)

print(performance_df)
print(t_test_results_df)
print(feature_importance_A)
print(feature_importance_B)

/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


  Time Horizon Metric  Experiment A  Experiment B  Improvement
0         y_t2     R²      0.146896      0.138524    -0.008372
1         y_t2    MSE      0.310142      0.313185     0.003044
2         y_t2    MAE      0.032693      0.033007     0.000314
3         y_t2   RMSE      0.556904      0.559629     0.002726
  Time Horizon     t-targe  T-statistic   P-value Significant
0         y_t2  Prediction     0.157112  1.750314          No
          Feature  Importance
47           剩余期限    0.074664
8      所属申万一级行业指数    0.026698
9        债券分类违约概率    0.024223
21        经营活动现金流    0.023640
46            成交量    0.016225
12           营业成本    0.012637
20         股东权益合计    0.012552
1       Shibor:3月    0.011818
33       资产净利率(%)    0.011230
15          非流动资产    0.011185
35    平均净资产收益率(%)    0.008236
25           流动比率    0.007503
11           营业收入    0.007211
42          授信剩余率    0.006544
4        PPI:当月同比    0.006155
16           资产总计    0.005566
7   社会融资规模存量:期末同比    0.004143
13           利润总额    

In [6]:
from tqdm import tqdm
import cudf
import cupy as cp
from cuml.model_selection import train_test_split
from cuml.ensemble import RandomForestRegressor
from cuml.metrics import r2_score, mean_squared_error, mean_absolute_error
from cupyx.scipy import special

def cupy_ttest_ind(a, b):
    # 确保输入是 cupy 数组
    if isinstance(a, float):
        a = cp.array([a])  # 转换为 cupy 数组
    if isinstance(b, float):
        b = cp.array([b])

    mean_a, mean_b = cp.mean(a), cp.mean(b)
    var_a, var_b = cp.var(a, ddof=1), cp.var(b, ddof=1)
    n_a, n_b = a.size, b.size

    pooled_se = cp.sqrt(var_a / n_a + var_b / n_b)
    t_stat = (mean_a - mean_b) / pooled_se
    df = (var_a / n_a + var_b / n_b) ** 2 / ((var_a / n_a) ** 2 / (n_a - 1) + (var_b / n_b) ** 2 / (n_b - 1))

    # 计算 p 值
    p_value = 2 * special.betainc(df / 2, 0.5, df / (df + t_stat**2))

    return t_stat, p_value

def permutation_importance(model, X, y, feature_names=None, metric=mean_squared_error, n_repeats=5):
    """
    计算基于置换的重要性
    参数:
    - model: 训练好的 cuML RandomForestRegressor
    - X: 特征矩阵 (CuPy array)
    - y: 目标值 (CuPy array)
    - feature_names: 特征名称 (列表或 cudf.Index)
    - metric: 评估指标 (默认 MSE)
    - n_repeats: 每个特征置换的重复次数
    
    返回:
    - feature_importances: 包含特征名和重要性的 cudf.DataFrame
    """
    # 确保 X 是 cupy.ndarray
    if isinstance(X, cudf.DataFrame):
        feature_names = X.columns  # 获取特征名称
        X = X.to_cupy()
    
    baseline_error = metric(y, model.predict(X))  # 计算基准 MSE
    importances = cp.zeros(X.shape[1])

    for col in range(X.shape[1]):
        scores = []
        for _ in range(n_repeats):
            X_permuted = X.copy()
            X_permuted[:, col] = X[:, col][cp.random.permutation(X.shape[0])]  # 置换该列
            permuted_error = metric(y, model.predict(X_permuted))
            scores.append(permuted_error - baseline_error)  # 误差增加量
        
        importances[col] = cp.mean(cp.array(scores))  # 计算均值
    
    # 确保特征名存在
    if feature_names is None:
        feature_names = [f"Feature {i}" for i in range(X.shape[1])]

    # 创建 cudf.DataFrame 存储特征重要性
    feature_importances = cudf.DataFrame({
        "Feature": feature_names,
        "Importance": importances.get()
    }).sort_values(by="Importance", ascending=False)  # 按重要性降序排列

    return feature_importances

cp.cuda.Device(0).use()
X_A = cudf.DataFrame.from_pandas(X_A)
X_B = cudf.DataFrame.from_pandas(X_B)
y = y_t3

performance_results = []
t_test_results = []

y = cp.array(y)
# 拆分数据集
X_A_train, X_A_test, y_train, y_test = train_test_split(X_A, y, test_size=0.2, random_state=42)
X_B_train, X_B_test = train_test_split(X_B, test_size=0.2, random_state=42)[0:2]  # 仅拆分 X_B

# 训练随机森林模型（实验 A）
rf_A = RandomForestRegressor(random_state=42)
rf_A.fit(X_A_train, y_train)
y_pred_A = rf_A.predict(X_A_test)

# 训练随机森林模型（实验 B）
rf_B = RandomForestRegressor(random_state=42)
rf_B.fit(X_B_train, y_train)
y_pred_B = rf_B.predict(X_B_test)

y_pred_A = cp.array(y_pred_A)
y_pred_B = cp.array(y_pred_B)

# 计算性能指标
r2_A = r2_score(y_test, y_pred_A)
mse_A = mean_squared_error(y_test, y_pred_A)
mae_A = mean_absolute_error(y_test, y_pred_A)
rmse_A = cp.sqrt(mse_A)

r2_B = r2_score(y_test, y_pred_B)
mse_B = mean_squared_error(y_test, y_pred_B)
mae_B = mean_absolute_error(y_test, y_pred_B)
rmse_B = cp.sqrt(mse_B)

# 存储性能差异
performance_results.append({
    "Time Horizon": "y_t3",
    "Metric": "R²",
    "Experiment A": r2_A,
    "Experiment B": r2_B,
    "Improvement": r2_B - r2_A
})
performance_results.append({
    "Time Horizon": "y_t3",
    "Metric": "MSE",
    "Experiment A": mse_A,
    "Experiment B": mse_B,
    "Improvement": mse_B - mse_A
})
performance_results.append({
    "Time Horizon": "y_t3",
    "Metric": "MAE",
    "Experiment A": mae_A,
    "Experiment B": mae_B,
    "Improvement": mae_B - mae_A
})
performance_results.append({
    "Time Horizon": "y_t3",
    "Metric": "RMSE",
    "Experiment A": rmse_A,
    "Experiment B": rmse_B,
    "Improvement": rmse_B - rmse_A
})

# t 检验
t_stat_pred, p_value_pred = cupy_ttest_ind(y_pred_A, y_pred_B)
t_test_results.append({
    "Time Horizon": "y_t3",
    "t-targe": "Prediction",
    "T-statistic": t_stat_pred,
    "P-value": p_value_pred,
    "Significant": "Yes" if p_value_pred < 0.05 else "No"
})


# 特征重要性
feature_importance_A = permutation_importance(rf_A, X_A_test, y_test)
feature_importance_B = permutation_importance(rf_B, X_B_test, y_test)

for entry in performance_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float

for entry in t_test_results:
    for key, value in entry.items():
        if isinstance(value, np.ndarray):  # NumPy 数组
            entry[key] = float(value.item())  # 转换为 Python float
        elif isinstance(value, cp.ndarray):  # CuPy 数组
            entry[key] = float(value.get().item())  # 先转 NumPy，再转 float


# 转换为 DataFrame
performance_df = cudf.DataFrame(performance_results)
t_test_results_df = cudf.DataFrame(t_test_results)

print(performance_df)
print(t_test_results_df)
print(feature_importance_A)
print(feature_importance_B)

/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:344: UserWarning: For reproducible results in Random Forest Classifier or for almost reproducible results in Random Forest Regressor, n_streams=1 is recommended. If n_streams is > 1, results may vary due to stream/thread timing differences, even when random_state is set
  return func(**kwargs)
/home/lyw/anaconda3/envs/rapids-24.12/lib/python3.10/site-packages/cuml/internals/api_decorators.py:188: UserWarning: To use pickling first train using float32 data to fit the estimator
  ret = func(*args, **kwargs)


  Time Horizon Metric  Experiment A  Experiment B  Improvement
0         y_t3     R²      0.250299      0.224771    -0.025529
1         y_t3    MSE      1.067359      1.103704     0.036345
2         y_t3    MAE      0.033224      0.033674     0.000451
3         y_t3   RMSE      1.033131      1.050573     0.017443
  Time Horizon     t-targe  T-statistic   P-value Significant
0         y_t3  Prediction      0.08929  1.857703          No
          Feature  Importance
47           剩余期限    0.222047
35    平均净资产收益率(%)    0.079663
30     有形净值债务率(%)    0.077196
33       资产净利率(%)    0.062046
20         股东权益合计    0.061844
13           利润总额    0.058850
1       Shibor:3月    0.057640
23        筹资活动现金流    0.046684
38        应收账款周转率    0.041083
22        投资活动现金流    0.040837
32       销售净利率(%)    0.039570
5        CPI:当月同比    0.033201
24           总现金流    0.030688
42          授信剩余率    0.029488
3   宏观经济景气指数:先行指数    0.026650
2          制造业PMI    0.023567
12           营业成本    0.022854
27          超速动比率    

In [7]:
# sum = 0
# for k, v in feature_importance_A.items():
#     sum += v

# print(sum)
type(feature_importance_A)

cudf.core.dataframe.DataFrame